In [1]:
!pip install xlsxwriter pydrive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 8.6 MB/s eta 0:00:00
  Created wheel for pydrive: filename=PyDrive-1.3.1-py3-none-any.whl size=27433 sha256=e078caf3d89df512a35344bf39591b2aa5ab317625ac0a631ed68a0c3eee4020
  Stored in directory: /root/.cache/pip/wheels/6c/10/da/a5b513f5b3916fc391c20ee7b4633e5cf3396d570cdd74970f
Successfully built pydrive


In [3]:
#%% import
import requests
from bs4 import BeautifulSoup
import json
import re


#%% functions

def get_sections_from_page(page_url):
    """
    Pobiera wszystkie sekcje/teksty z jednej strony HTML
    Każda sekcja ma swój anchor (#t5, #t6 etc.)
    """
    try:
        r = requests.get(page_url, timeout=10)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        sections = []

        # Znajdź spis treści
        spis = soup.find(id='spis')
        if not spis:
            # Może być w innym miejscu
            spis = soup.find('h2', string=lambda x: x and 'SPIS' in x)

        if spis:
            print(f"✓ Znaleziono spis treści")
            # Zbierz wszystkie linki z kotwicom (#t5, #t6 etc.)
            parent = spis.find_parent()
            if not parent:
                parent = spis.find_next_sibling()

            if parent:
                links = parent.find_all('a', href=True)
                for link in links:
                    href = link['href']
                    title = link.get_text(strip=True)

                    # Pomijamy link do spisu
                    if href == '#spis' or 'SPIS' in title:
                        continue

                    # Linki z kotwicami (#t5, #t6...)
                    if href.startswith('#'):
                        section_id = href[1:]  # usuń #
                        full_url = f"{page_url}{href}"
                        sections.append({
                            'id': section_id,
                            'title': title,
                            'url': full_url,
                            'anchor': href
                        })

        print(f"  Znaleziono {len(sections)} sekcji")
        return sections

    except Exception as e:
        print(f"Błąd: {e}")
        return []


def find_all_pages():
    """
    Znajduje wszystkie strony z wierszami/tekstami
    """
    base_url = "https://miloszbiedrzycki.pl/"

    print("Szukam stron z wierszami...")

    # Lista znanych stron (może być więcej)
    pages_to_check = [
        f"{base_url}pyllyp.html",
        f"{base_url}index.html",
        # Dodaj tutaj inne strony jeśli są
    ]

    all_sections = []

    for page_url in pages_to_check:
        print(f"\nSprawdzam: {page_url}")
        try:
            sections = get_sections_from_page(page_url)
            if sections:
                # Dodaj nazwę strony do każdej sekcji
                page_name = page_url.split('/')[-1].replace('.html', '')
                for section in sections:
                    section['page'] = page_name
                    section['page_url'] = page_url
                all_sections.extend(sections)
        except Exception as e:
            print(f"  Błąd: {e}")

    return all_sections


#%% main execution

if __name__ == "__main__":
    print("="*60)
    print("Pobieranie sekcji z miloszbiedrzycki.pl")
    print("="*60)
    print("UWAGA: To nie są osobne pliki, tylko sekcje na stronie!\n")

    # Pobierz wszystkie sekcje
    all_sections = find_all_pages()

    if not all_sections:
        print("\n⚠️  Nie znaleziono sekcji!")
        print("Sprawdź ręcznie strony.")
        exit(1)

    print(f"\n{'='*60}")
    print(f"RAPORT")
    print(f"{'='*60}")
    print(f"Znaleziono sekcji/wierszy: {len(all_sections)}")

    # Pokaż przykłady
    print("\nPrzykładowe sekcje (pierwsze 10):")
    for i, section in enumerate(all_sections[:10], 1):
        print(f"  {i}. [{section['page']}] {section['title']} -> {section['anchor']}")

    if len(all_sections) > 10:
        print(f"  ... i {len(all_sections) - 10} więcej")

    # Zapisz do JSON
    output_data = {
        'source': 'https://miloszbiedrzycki.pl/',
        'note': 'Sekcje na stronach, nie osobne pliki',
        'total_sections': len(all_sections),
        'sections': all_sections
    }

    with open('biedrzycki_sekcje.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    print(f"\nZapisano do:")
    print(f"  - biedrzycki_sekcje.json")


Pobieranie sekcji z miloszbiedrzycki.pl
UWAGA: To nie są osobne pliki, tylko sekcje na stronie!

Szukam stron z wierszami...

Sprawdzam: https://miloszbiedrzycki.pl/pyllyp.html
✓ Znaleziono spis treści
  Znaleziono 47 sekcji

Sprawdzam: https://miloszbiedrzycki.pl/index.html
  Znaleziono 0 sekcji

RAPORT
Znaleziono sekcji/wierszy: 47

Przykładowe sekcje (pierwsze 10):
  1. [pyllyp] światło tu -> #t5
  2. [pyllyp] północ rozlewiste -> #t6
  3. [pyllyp] filmowy pociąg -> #t8
  4. [pyllyp] światło powoli -> #t9
  5. [pyllyp] jedna lokomotywa -> #t10
  6. [pyllyp] rozrzutność ciał -> #t11
  7. [pyllyp] jadę tyłem -> #t12
  8. [pyllyp] emaliowane czarne -> #t13
  9. [pyllyp] handluję karabinami -> #t14
  10. [pyllyp] bordowe zmierzchy -> #t15
  ... i 37 więcej

Zapisano do:
  - biedrzycki_sekcje.json

💡 UWAGA: Użyj specjalnego scrapera dla sekcji,
   nie standardowego scrapera dla osobnych stron!


In [4]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import json
import xlsxwriter


#%% functions

def scrape_section(page_url, section_id, section_title):
    """
    Pobiera zawartość konkretnej sekcji ze strony
    """
    try:
        r = requests.get(page_url, timeout=15)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'lxml')

        # Znajdź element z tym ID
        section_elem = soup.find(id=section_id)

        if not section_elem:
            return None

        # Zbierz tekst od tej sekcji do następnej (lub końca)
        text_parts = []
        current = section_elem

        # Zbierz wszystkie elementy między tą kotwicą a następną
        while current:
            current = current.find_next_sibling()
            if not current:
                break

            # Jeśli trafimy na następną kotwicę (nową sekcję), stop
            if current.name == 'a' and current.get('id'):
                break

            # Zbierz tekst
            if current.name in ['p', 'div', 'span']:
                text = current.get_text(strip=True)
                if text:
                    text_parts.append(text)

        full_text = ' '.join(text_parts)

        return {
            'Link': f"{page_url}#{section_id}",
            'Data publikacji': "no date",  # Strony statyczne rzadko mają daty
            'Tytuł artykułu': section_title,
            'Tekst artykułu': full_text,
            'Autor': "Miłosz Biedrzycki",
            'Kategoria': "wiersz",  # To wiersze
            'Linki zewnętrzne': None,
            'Zdjęcia/Grafika': False,
            'Filmy': False,
            'Linki do zdjęć': None
        }

    except Exception as e:
        print(f"Błąd dla sekcji {section_id}: {e}")
        return None


#%% main execution

if __name__ == "__main__":
    # Wczytaj sekcje z JSON
    try:
        with open('biedrzycki_sekcje.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
            sections = data['sections']
        print(f"Wczytano {len(sections)} sekcji z pliku")
    except FileNotFoundError:
        print("Nie znaleziono pliku biedrzycki_sekcje.json")
        print("Użyj najpierw get_biedrzycki_links.py!")
        exit(1)

    all_results = []
    errors = []

    print("\n" + "="*60)
    print("Rozpoczynam scraping sekcji z miloszbiedrzycki.pl")
    print("="*60 + "\n")

    # Grupuj sekcje po stronie (żeby nie pobierać tej samej strony wielokrotnie)
    pages_sections = {}
    for section in sections:
        page_url = section['page_url']
        if page_url not in pages_sections:
            pages_sections[page_url] = []
        pages_sections[page_url].append(section)

    # Scrapuj każdą stronę raz i wyciągnij wszystkie sekcje
    for page_url, page_sections in tqdm(pages_sections.items(), desc="Przetwarzanie stron"):
        try:
            # Pobierz stronę raz
            r = requests.get(page_url, timeout=15)
            r.encoding = 'utf-8'
            soup = BeautifulSoup(r.text, 'lxml')

            # Przetwórz wszystkie sekcje z tej strony
            for section in page_sections:
                section_id = section['id']
                section_title = section['title']

                # Znajdź element
                section_elem = soup.find(id=section_id)

                if not section_elem:
                    errors.append(f"{page_url}#{section_id}")
                    continue

                # Zbierz tekst
                text_parts = []
                current = section_elem

                while current:
                    current = current.find_next_sibling()
                    if not current:
                        break
                    if current.name == 'a' and current.get('id'):
                        break
                    if current.name in ['p', 'div', 'span', 'pre']:
                        text = current.get_text(strip=True)
                        if text:
                            text_parts.append(text)

                full_text = ' '.join(text_parts)

                result = {
                    'Link': f"{page_url}#{section_id}",
                    'Data publikacji': "no date",
                    'Tytuł artykułu': section_title,
                    'Tekst artykułu': full_text,
                    'Autor': "Miłosz Biedrzycki",
                    'Kategoria': "wiersz",
                    'Linki zewnętrzne': None,
                    'Zdjęcia/Grafika': False,
                    'Filmy': False,
                    'Linki do zdjęć': None
                }

                all_results.append(result)

            time.sleep(0.3)  # Ostrożność

        except Exception as e:
            print(f"Błąd dla {page_url}: {e}")
            errors.append(page_url)

    # Zapisywanie wyników
    timestamp = datetime.today().date()

    # JSON
    with open(f'biedrzycki_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    # Excel
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(f"biedrzycki_{timestamp}.xlsx",
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)

    # Raport
    print(f"\n{'='*60}")
    print(f"Scraping zakończony!")
    print(f"Przetworzono sekcji/wierszy: {len(all_results)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nProblematyczne sekcje (pierwsze 10):")
        for error in errors[:10]:
            print(f"  - {error}")
    print(f"\nPliki wyjściowe:")
    print(f"  - biedrzycki_{timestamp}.json")
    print(f"  - biedrzycki_{timestamp}.xlsx")
    print(f"{'='*60}\n")

Wczytano 47 sekcji z pliku

Rozpoczynam scraping sekcji z miloszbiedrzycki.pl



Przetwarzanie stron: 100%|██████████| 1/1 [00:01<00:00,  1.09s/it]


Scraping zakończony!
Przetworzono sekcji/wierszy: 47
Błędów: 0

Pliki wyjściowe:
  - biedrzycki_2026-01-12.json
  - biedrzycki_2026-01-12.xlsx




/tmp/ipython-input-2233669984.py:170: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)


In [5]:
df.head()

,Link,Data publikacji,Tytuł artykułu,Tekst artykułu,Autor,Kategoria,Linki zewnętrzne,Zdjęcia/Grafika,Filmy,Linki do zdjęć
0,https://miloszbiedrzycki.pl/pyllyp.html#t5,no date,światło tu,"...światło, tu jako pojedynczakorpuskuła, mała...",Miłosz Biedrzycki,wiersz,None,False,False,None
1,https://miloszbiedrzycki.pl/pyllyp.html#t6,no date,północ rozlewiste,"Północ, rozlewiste czerwcowezmierzchywyciągają...",Miłosz Biedrzycki,wiersz,None,False,False,None
2,https://miloszbiedrzycki.pl/pyllyp.html#t8,no date,filmowy pociąg,filmowy pociąg wjeżdża na stację po szynach pe...,Miłosz Biedrzycki,wiersz,None,False,False,None
3,https://miloszbiedrzycki.pl/pyllyp.html#t9,no date,światło powoli,"światło powoli wycieka, coraz łatwiej uwierzyć...",Miłosz Biedrzycki,wiersz,None,False,False,None
4,https://miloszbiedrzycki.pl/pyllyp.html#t10,no date,jedna lokomotywa,"jedna lokomotywa z przodu składu, drugaz tyłu ...",Miłosz Biedrzycki,wiersz,None,False,False,None
